In [1]:
import os
import zipfile
import gdown
from pathlib import Path
from dataclasses import dataclass
from cnnClassifier import logger
from cnnClassifier.constants import *
from cnnClassifier.utils.helper import read_yaml, create_directories, get_size

In [2]:
@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [3]:
class ConfigurationManager:
    def __init__(
        self,
        # config_filepath = CONFIG_FILE_PATH,       
        # params_filepath = PARAMS_FILE_PATH

        config_filepath = '../config/config.yaml',
        params_filepath = '../params.yaml'

    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])


    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config

In [4]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        try:
            url = self.config.source_URL
            dwn_dir = self.config.local_data_file
            os.makedirs('artifacts/data_ingestion', exist_ok=True)
            logger.info('Downloading file from %s to %s', url, dwn_dir)

            file_id = url.split('/')[-2]
            download_url = f"https://drive.google.com/uc?/export=download&id={file_id}"

            gdown.download(download_url, dwn_dir )

            logger.info('Download completed')
        
        except Exception as e:
            raise e
    
    def extractor(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as f:
            f.extractall(unzip_path)

In [5]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extractor()

except Exception as e:
    raise e

[2024-06-12 01:41:25,035: INFO: helper: yaml file: ../config/config.yaml loaded successfully]
[2024-06-12 01:41:25,035: INFO: helper: yaml file: ../params.yaml loaded successfully]
[2024-06-12 01:41:25,043: INFO: helper: Created directory at: artifacts]
[2024-06-12 01:41:25,048: INFO: helper: Created directory at: artifacts/data_ingestion]
[2024-06-12 01:41:25,049: INFO: 391097943: Downloading file from https://drive.google.com/file/d/1xb_Q9FiQ9X33h99R7LTkdDeSXG_E9hty/view?usp=drive_link to artifacts/data_ingestion/data.zip]


Downloading...
From (original): https://drive.google.com/uc?/export=download&id=1xb_Q9FiQ9X33h99R7LTkdDeSXG_E9hty
From (redirected): https://drive.google.com/uc?%2Fexport=download&id=1xb_Q9FiQ9X33h99R7LTkdDeSXG_E9hty&confirm=t&uuid=b7185638-873e-411b-8660-1d257b1875b4
To: c:\Users\jaksh\Desktop\Programs\Projects\knee-arthritis-detection\experiments\artifacts\data_ingestion\data.zip
100%|██████████| 61.1M/61.1M [00:08<00:00, 6.80MB/s]


[2024-06-12 01:41:35,487: INFO: 391097943: Download completed]
